In [1]:
%load_ext autoreload
%autoreload 2


## classic pydata stack
import os 
import numpy as np
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf

%matplotlib inline 

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (15,7)


In [2]:
import sys

import vanilla_NN as NN

sys.path.append('..')
from pipeline import *
from models import *


In [3]:
def a():
    return 1,2

In [4]:
l=[4,*a(),5]
print(l)

[4, 1, 2, 5]


In [43]:
pipe = PairSingle_Pipeline(num_blocks=6)
dataset = PolymerDataset(data_paths=["../data/AA66266AA.npy","../data/AA662266AA.npy"],pipeline=pipe)

Processing ../data/AA662266AA.npy: 100%|██████████| 17625/17625 [00:49<00:00, 359.14it/s]


In [44]:
train_data, test_data = train_test_split(dataset)


In [45]:
model = VanillaLSTM(input_dim=dataset.num_features, output_dim=dataset.num_classes, num_layers=1, hidden_dim=64)

In [51]:
model, test_metrics = train(train_data, model, batch_size=512, num_epochs=200, lr_rate=0.005)

2021-12-20 16:42:15.626977: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-20 16:42:15.626998: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


epoch=0/30, loss=0.10845699613647801, accuracy=95.88301849365234
epoch=1/30, loss=0.10912274822060551, accuracy=95.85462951660156
epoch=2/30, loss=0.10495774421308722, accuracy=95.87947082519531
epoch=3/30, loss=0.10428129143214651, accuracy=95.91141510009766
epoch=4/30, loss=0.10404094263711679, accuracy=95.94690704345703
epoch=5/30, loss=0.10395806796649205, accuracy=95.91851043701172
epoch=6/30, loss=0.10833293399108308, accuracy=95.92205810546875
epoch=7/30, loss=0.10485011598627482, accuracy=95.91141510009766
epoch=8/30, loss=0.10548055065529686, accuracy=95.9540023803711
epoch=9/30, loss=0.10704067801790577, accuracy=95.92205810546875
epoch=10/30, loss=0.1047423624300531, accuracy=95.98594665527344
epoch=11/30, loss=0.10847828696881022, accuracy=95.97174835205078
epoch=12/30, loss=0.10542138898745179, accuracy=95.9362564086914
epoch=13/30, loss=0.10399722258028175, accuracy=95.97529602050781
epoch=14/30, loss=0.10402360096174691, accuracy=96.0001449584961
epoch=15/30, loss=0.1060

In [52]:
test(test_data,model=model)

{'accuracy': 0.9532869515831321,
 'f1_score': 0.9536815430099959,
 'precision': 0.9589467723669309,
 'recall': 0.948473816858023,
 'confusion_matrix': array([[0.958, 0.042],
        [0.052, 0.948]])}

In [11]:
np.save(arr=dataset.labels.numpy(),file="backbone_bulky4_labels")

In [12]:
def kfold_cv_iter(y: np.ndarray, tx: np.ndarray, k: int = 5, seed: float = 1):
    """K-fold cross validation. Split data into k parts and iterate through the folds
    Args:
        y (np.ndarray): Label data
        tx (np.ndarray): Features data
        k (int, optional): Number of folds. Defaults to 5.
        seed (float, optional): Seed for randomization. Defaults to 1.
    Yields:
        Generator: (x_train, y_train, x_test, y_test)
    """
    num_row = y.shape[0]
    fold_size = int(num_row / k)
    np.random.seed(seed)
    indices = np.random.permutation(num_row)
    
    for i in range(k):
        test_indices = indices[i * fold_size: (i + 1) * fold_size]
        train_indices = list(set(range(num_row)) - set(test_indices))
        yield tx[train_indices], y[train_indices], tx[test_indices], y[test_indices]


In [13]:
print(dataset.full_dataset.numpy())

AttributeError: 

In [ ]:
for i in dataset.full_dataset
    print(i)

0


In [ ]:
mask = np.zeros((9), dtype=bool)

mask[0] = True

test = tx[:,np.logical_not(mask)]


NameError: name 'tx' is not defined

In [ ]:
tx[0]

array([ 3.51094612,  3.76399497, -0.74496833,  3.24895359,  0.67451326,
       -0.94954394, -0.92173372, -0.89233789,  1.90161115])

In [ ]:
y = np.load("backbone_bulky4_labels.npy",allow_pickle=True)

## Random forest with removing a feature

In [ ]:
names=["mean", "median", "std", "min", "max", "len", "count_extremums", "max_slope", "min_slope"]

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

for i in range(9):
    tx = np.load("backbone_bulky4_data_1.npy",allow_pickle=True)
    tx = tx.reshape((tx.shape[0],-1))
    mask = np.zeros((9), dtype=bool)
    mask[i] = True

    tx =tx[:,np.logical_not(mask)]


    iter = kfold_cv_iter(y=y,tx=tx,k=3,seed=2)
    accuracies = []
    for X_train, Y_train, X_test, Y_test in iter:
        classifier = RandomForestClassifier(random_state=89,n_estimators = 100 )
        classifier.fit(X_train, Y_train)
        y_pred = classifier.predict(X_test)
        accuracy = accuracy_score(Y_test, y_pred)
        accuracies.append(accuracy)

    print(names[i],": ",np.array(accuracies).mean())


FileNotFoundError: [Errno 2] No such file or directory: 'backbone_bulky4_data_1.npy'

## Random forest with different num_blocks

In [ ]:
for i in range(1,11):
    path = f"backbone_bulky4_data_{i}.npy"
    tx = np.load(path,allow_pickle=True)
    tx = tx.reshape((tx.shape[0],-1))

    iter = kfold_cv_iter(y=y,tx=tx,k=3,seed=2)
    accuracies = []
    for X_train, Y_train, X_test, Y_test in iter:
        classifier = RandomForestClassifier(random_state=89,n_estimators = 100 )
        classifier.fit(X_train, Y_train)
        y_pred = classifier.predict(X_test)
        accuracy = accuracy_score(Y_test, y_pred)
        accuracies.append(accuracy)

    print(f"num_blocks={i}",": ",np.array(accuracies).mean())

FileNotFoundError: [Errno 2] No such file or directory: 'backbone_bulky4_data_1.npy'

### Gradient boosting classifier with removing a feature

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

names=["mean", "median", "std", "min", "max", "len", "count_extremums", "max_slope", "min_slope"]
from sklearn.metrics import accuracy_score

for i in range(9):
    tx = np.load("backbone_bulky4_data_1.npy",allow_pickle=True)
    tx = tx.reshape((tx.shape[0],-1))
    mask = np.zeros((9), dtype=bool)
    mask[i] = True

    tx =tx[:,np.logical_not(mask)]


    iter = kfold_cv_iter(y=y,tx=tx,k=3,seed=2)
    accuracies = []
    for X_train, Y_train, X_test, Y_test in iter:
        classifier =GradientBoostingClassifier(random_state=89,n_estimators = 100 )
        classifier.fit(X_train, Y_train)
        y_pred = classifier.predict(X_test)
        accuracy = accuracy_score(Y_test, y_pred)
        accuracies.append(accuracy)

    print(names[i],": ",np.array(accuracies).mean())


mean :  0.9448266130454721
median :  0.9422866617278723
std :  0.9335379405228067
min :  0.9263766888912407
max :  0.9379475782269728
len :  0.9400289272233393
count_extremums :  0.9446855046389389
max_slope :  0.9445443962324056
min_slope :  0.9424630472360391


 ## Gradient boosting classifier with different num_blocks

In [ ]:
for i in range(1,11):
    path = f"backbone_bulky4_data_{i}.npy"
    tx = np.load(path,allow_pickle=True)
    tx = tx.reshape((tx.shape[0],-1))

    iter = kfold_cv_iter(y=y,tx=tx,k=3,seed=2)
    accuracies = []
    for X_train, Y_train, X_test, Y_test in iter:
        classifier = GradientBoostingClassifier(random_state=89,n_estimators = 100 )
        classifier.fit(X_train, Y_train)
        y_pred = classifier.predict(X_test)
        accuracy = accuracy_score(Y_test, y_pred)
        accuracies.append(accuracy)

    print(f"num_blocks={i}",": ",np.array(accuracies).mean())

num_blocks=1 :  0.944050516809539
num_blocks=2 :  0.9525522983031713
num_blocks=3 :  0.9549864183158712
num_blocks=4 :  0.9551275267224044
num_blocks=5 :  0.9546336472995378
num_blocks=6 :  0.9557625145518044
num_blocks=7 :  0.9553391893322045
num_blocks=8 :  0.9553391893322045
num_blocks=9 :  0.9547747557060712
num_blocks=10 :  0.944050516809539


In [ ]:
class PaddedDataset():

    def __init__(self, data_paths,seed=10):
        self.process(data_paths,seed)


    def process(self,data_paths,seed):

        raw_data = [np.load(data_path, allow_pickle=True) for data_path in data_paths]
        labels = [0,1]

    ## balance the dataset by removing signals that are too short or too long
    ## first we build the dataframe to know the lengths of the time series

        len_series = []

        for data in raw_data:
            lengths= []
            for row in data:
                ## length of time series
                lengths.append(row.shape[0])

            len_series.append(pd.Series(lengths))

        ## enforces that the first dataset is the smaller one in total size
        ## such that we can apply our balancing operations generally
        if len(len_series[0]) > len(len_series[1]):
            len_series.reverse()
            raw_data.reverse()
            labels.reverse()

        ## filter the dataset and remove signals that are:
        ## too short i.e. < len_series[0].quantile(0.1)
        ## too long i.e. > len_series[0].quantile(0.9)
        for i in range(2):
            mask = (len_series[i] > max(len_series[0].quantile(0.1),num_blocks)) & (len_series[i] < len_series[0].quantile(0.5))
            raw_data[i] = raw_data[i][mask]

        ## most likely, one dataset is still bigger than the other one
        ## therefore, we randomly sample data from the bigger dataset to create a new dataset of the same size as the small one 
        np.random.seed(seed=seed)

        # making sure the smallest dataset is the first one
        if len(raw_data[0]) > len(raw_data[1]):
            raw_data.reverse()
            labels.reverse()

        # randomly sampling and making a balanced dataset
        raw_data[1]  = np.random.permutation(raw_data[1])[:len(raw_data[0])]
        data=[]
        data_labels=[]
        
        max_length = max(len_series[0].max(), len_series[1].max())
        print("Max length",max_length)

        ## using our Input class to build the entire dataset and extracting features from each row
        for index, raw_data in enumerate(raw_data):
            for raw_series in raw_data:
                padded_series = self.pad(raw_series,max_length)
                data.append(padded_series)
                data_labels.append(labels[index])
        data = np.array(data)

        #normalizing features
        data = (data - data.mean(axis=0)) / data.std(axis=0)
        data_labels = np.array(data_labels)

        self.data = data
        self.labels = data_labels

        return self

    
    def pad(self,raw_series, max_length):

        l = max_length - raw_series.shape[0]
        x=raw_series[:,0]
        y= raw_series[:,1]

        if l > 0:
            res1 = np.concatenate((x, np.linspace(start= x[-1] + 0.01, stop= x[-1] + l*0.01 , num=l)),axis=None)
            res2 = np.concatenate((y, np.zeros(l)),axis=None)

        res = np.vstack((res1,res2)).T

        return res
        

In [ ]:
dataset = PaddedDataset(data_paths=["AA00400AA.npy","AA66466AA.npy"])

MemoryError: 

In [ ]:
from sktime.classification.interval_based import TimeSeriesForestClassifier
from sktime.datasets import load_arrow_head
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X, y = dataset.data, dataset.labels
X_train, X_test, y_train, y_test = train_test_split(X, y)
classifier = TimeSeriesForestClassifier()
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
accuracy_score(y_test, y_pred)

ValueError: If passed as a np.array, X must be a 3-dimensional array, but found shape: (21243, 27)

In [ ]:
dataset.data[1]

array([ 0.17944997, -0.20482437,  0.67186573, -0.00658525,  0.94489608,
       -0.81025791, -0.73619943,  1.31484999, -0.43366814, -0.71692991,
       -0.57235341, -0.45231736, -0.82291628, -0.43769871, -0.81025791,
       -0.89015383,  1.00854561, -1.42100356, -0.48263948, -1.11383077,
        0.41667791, -0.81664598,  0.96985427, -0.81025791, -0.71873826,
        1.37042307, -0.68925605])